# Introduction

The goal of RMI is to have a middleware make life for programming in DS easier. We want to create a platform for programming over DS, a little bit easier than socket programming. We're designing a protocol to do that, without explicitly referring to sockets.

Two widely used models are:
* Remote Procedure Call - an extension of the conventional procedure call model, pretty much de-functioned now
* Remote Method Invocation - an extension of the object-oriented programming model

<img src="img/img33.png" width="500">

# Request-Reply Protocol

We want to generalize a request. This is an exchange protocol for the implementation of remote invocation in a distributed system.

We discuss the protocol based on three abstract operations: $doOperation, getRequest, sendReply$.

<img src="img/img34.png" width="500">

## Request-Reply Operations

```
public byte[] doOperation(RemoteRef s, int operationId, byte[] arguments)
```

* Sends a request message to the remote server and returns the reply
* The arguments specify the remote server, the operation to be invoked and the arguments of that operation
* RemoteRef represents the communication information may include port, socket and IP address

```
public byte[] getRequest()
```

* Acquires a client request via the server port

```
public void sendReply(byte[] reply, InetAddress clientHost, int clientPort)
```

* Sends the reply message to the client at its Internet address and port

A message in a request-reply protocol typically contains a number of fields as shown below:

<img src="img/img35.png" width="500">

requestId - which number is associated with which call. If the request is repeated or there is a communication problem, you need an unique id to figure out is this a repeat request or same request and what happened.

## Failure Model

If operations are implemented using UDP, then they suffer from the same communication failures:
* Lost messages
* Out of order - the remote method you are going to call may be a huge structure, you can't pass it with a single value or fit into a simple datagram, you have to divide it into pieces.

In addition, the protocol can suffer from the failure of processes(crash failures)

## Design Issues

Fault tolerance measures we can consider include:
* Timeouts - It's only a approximate for failure, do I try again or give an exception
* Discarding duplicate messages
* Handling lost reply messages - strategy depends on whether the server operations are idemponent(an operation that can be performed repeatedly)
* History - This identifier request this operation, if the same identifier same request comes in again, you may ignore, just send the reply, don't repeat the operation. If servers have to send replies without re-execution, a history has to be maintained

Three main design decisions related to implementations of the request/reply protocols are:
* Strategy to retry request message
* Mechanism to filter duplicates
* Strategy for results retransmission

## Exchange Protocols

Three different types of protocols are typically used that address the design issues to a varying degree depends on the application:
* The request(R) protocol - can be used for video processing
* The request-reply(RR) protocol
* The request-reply-acknowledge reply(RRA) protocol

You can still acknowledge an acknowledgment, but this is endless regardless how many replies and requests you have, you can't go through the barrier, this is called impossibility of perfect agreement on asynchronous system.

Middleware that implements remote invocation generally provides a certain level of semantics:
* **Maybe**: The remote procedure call may be executed once or not at all. Unless the caller receives a result, it is unknown as to whether the remote procedure was called.
* **At-least-once**: Either the remote procedure was executed at least once, and the caller received a response, or the caller received an exception to indicate the remote procedure was not executed at all.
* **At-most-once**: The remote procedure call was either executed exactly once, in which case the caller received a response, or it was not executed at all and the caller receives an exception.

Java RMI supports at-most-once invocation. Sun RPC supports at-least-once semantics.

<img src="img/img36.png" width="500">

If you have some faults, and you want to make sure that maybe your request reached the other side like the video streaming example, you don't need to deal with re-transmission/re-execution.

If your application is idemponent, you may want to do it in at-least-once way.

## Client-Server Communication

Client-Server communication normally uses the synchronous request-reply communication paradigm. It involves send and receive operations.

TCP or UDP(no reply) can be used - TCP involves additional overheads:
* Redundant acknowledgments
* Needs additional messages for establishing connection
* Flow control is not needed since the number of arguments and results are limited

Longevity of TCP connections is an important design issue

# Java RMI

## Object Oriented Concepts

Object References:
* Used to access objects
* Object references can be assigned to variables, passed as arguments and returned as results

Interfaces:
* Define the methods that are available to objects to invoke
* Each method signature specifies the arguments and return values

## Distributed Object Concepts

Remote Object
* An object that can receive remote invocations
* Can also receive local invocations
* Can invoke methods in local objects as well as other remote objects

<img src="img/img37.png" width="500">

Remote Object Reference
* An identifier that can be used throughout a distributed system to refer to a particular unique remote object
* Other objects can invoke the methods of a remote object if they have access to its remote object reference

<img src="img/img38.png" width="500">

time in the table refers to time of creation of particular object(to be an unique reference for an object), object number can be the identifier name.

Remote Interface
* Defines the methods that can be invoked by external processes
* Remote objects implement the remote interface

<img src="img/img39.png" width="500">

Actions
* Actions can be performed on remote objects
* An action could be executing a remote method defined in the remote interface or creating a new object in the target process
* Actions are invoked using RMI

<img src="img/img40.png" width="500">

## Implementation of RMI

RMI is composed of:
* communication module
* remote reference module
* the RMI software

<img src="img/img41.png" width="500">

### Communication Module

The two cooperating communication modules carry out the request-reply protocol.

It uses three fields from the message:
* Message type
* Request ID
* Remote object reference

It is responsible for implementing the invocation semantics.

The server module queries the remote reference module(got from communication module) to obtain the local reference of the object and passes the local reference to the dispatcher to access certain class.

### Remote Reference Module

It is responsible for translating between local and remote object references and for creating remote object references.

It has a remote object table that records the correspondence between local object references in that process and remote object references(system-wide).

### RMI Software

Software layer that lies between the application and the communication, object reference modules.

Proxy: The role of the proxy is to make RMI transparent to clients by behaving like a local object to the invoker, but actually contacting the remote object. There is a proxy for each remote object which is responsible for:
* Marshalling the reference of the target object, its own method id and the arguments and forwarding them to the communication module
* Unmarshalling the results and forwarding them to the invoking object

Dispatcher: There is one dispatcher for each remote object class. It receives the request message from the communication module and uses the methodId to select the appropriate method in the skeleton, passing on the request message.

Skeleton: It acts as the interface of the remote object. Unmarshalling the arguments in the request and forwarding them to the servant(objects in the process that receive the remote invocation). Marshalling the results from the servant to be returned to the client.

## Implementing an RMI Application

Remote Interface:
* Exposes the set of methods and properties available
* Defines the contract between the client and server
* Constitutes the root for both stub and skeleton

Servant component:
* Represents the remote object
* Implements the remote interface

Server component:
* Main driver that creates, initializes, and makes available the servant
* Registers the servant with the binder

Client component:
* Uses the binder to lookup for remote objects

Compile source and generate stubs:
* Client proxy or stub
* Server dispatcher and skeleton

## The Binder

Client programs require a way to obtain the remote object reference of the remote objects in the server. A binder is a service in a distributed system that supports this functionality. A binder maintains a table containing mappings from textual names to object references. Servers register their remote objects(by name) with the binder. Clients look them up by name.

## Transparency

Although **location** and **access** transparency are goals for remote invocation, in some cases complete transparency is not desirable.

Remote invocations are more vulnerable to failures:
* They involve a network, potentially another computer
* There is always the chance that no result is received
* Clients making remote invocations should be able to recover from this situation

The latency of a remote invocation is much larger than that of a local one. Programs making remote invocations should be able to take this into account so that they can:
* Minimize the number of remote invocations
* Abort a remote invocation that is taking too long

Therefore, many implementations provide location transparency but not complete access transparency. This enables the programmer to make optimization decisions.

<img src="img/img42.png" width="500">

## Code

Specify the Remote Interface

```
public interface IRemoteMath extends Remote {
    double add(double i, double j) throws RemoteException;
    double subtract(double i, double j) throws RemoteException;
}
```

Implement the Servant Class. Whenever you create an object of the UnicastRemoteObject, it pushes the object into RMI registry. If you don't extend this class, you have to manually push it into the RMI registry.

```
public class RemoteMathServant extends UnicastRemoteObject implements IRemoteMath {
    public double add(double i, double j) throws RemoteException {
        return (i+j);
    }
    
    public double subtract(double i, double j) throws RemoteException {
        return (i-j);
    }
}
```

Implement the server

```
public class MathServer {
    public static void main(String args[]) {
        System.setSecurityManager(new RMISecurityManager());
        try {
            IRemoteMath remoteMath = new RemoteMathServant();
            Registry registry = LocateRegistry.getRegistry();
            registry.bind("Compute", remoteMath);
            System.out.println("Math server ready");
        } catch(Exception e) {
            e.printStackTrace();
        }
    }
}
```

Implement the client program

```
public class MathClient {
    public static void main(String args[]) {
        try {
            if(System.getSecurityManager() == null)
                System.setSecurityManager(new RMISecurityManager());
                
            LocateRegistry.getRegistry("localhost");
            IRemoteMath math = (IRemoteMath) registry.lookup("Compute");
            
            System.out.println("1.7 + 2.8 = " + math.add(1.7, 2.8));
        } catch(Exception e) {
            e.printStackTrace();
        }
    }
}
```

# Remote Procedure Call

RPCs enable clients to execute procedures in server processes based on a defined service interface.

<img src="img/img43.png" width="500">

Communication module:
* Implements the desired design choices in terms of retransmission of requests, dealing with duplicates and retransmission of results

Client Stub Procedure:
* Behaves like a local procedure to the client. Marshalls the procedure identifier and arguments which is handed to the communication module
* Unmarshalls the results in the reply

Dispatcher:
* Selects the server stub based on the procedure identifier and forwards the request to the server stub

Server Stub Procedure:
* Unmarshalls the arguments in the request message and forwards it to the service procedure
* Marshalls the arguments in the result message and returns it to the client